In [1]:
# 在指定长度附近截取字符串的函数
def clip(text, max_len=80):
    """在 max_len 前面或后面的第一个空格处截断文本"""
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.find(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:
        end = len(text)
    return text[:end].rstrip()

参数名称在 \_\_code__.co_varnames 中，不过里面还有函数定义体中创建的局部变量。  
因此，参数名为前 N 个字符串，而 N 的值由 \_\_code__.co_argcount 确定。  
这里不包含前缀为 * 或 ** 的变长参数。  
参数的默认值只能通过他们在 \_\_defaults__ 元组中的位置确定。因此要从后往前扫描才能把参数和默认值对应起来  
在这个示例中 clip 函数有两个参数，text 和 max_len，其中一i给有默认值，为 80，因此它必然属于最后一个参数

In [6]:
# 提取关于函数参数的信息
print(clip.__defaults__)
print(clip.__code__)
print(clip.__code__.co_varnames)
print(clip.__code__.co_argcount)

(80,)
<code object clip at 0x000002B2CBBA6C90, file "C:\Users\16052\AppData\Local\Temp/ipykernel_7676/1361252703.py", line 2>
('text', 'max_len', 'end', 'space_before', 'space_after')
2


In [7]:
# 默认值与参数名顺序相反并不容易使用，但我们有 inspect 模块
# 提取函数的签名
from inspect import signature
sig = signature(clip)
sig

<Signature (text, max_len=80)>

In [8]:
for name, param in sig.parameters.items():
    print(param.kind, ':', name, '=', param.default)

POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


inspect.Parameter.kind 的属性可以是以下五个值之一：
* POSITIONAL_OR_KEYWORD 可以通过定位参数和关键字参数传入的形参
* VAR_POSITIONAL        定位参数元组
* VAR_KEYWORD           关键字参数字典
* KEYWORD_ONLY          仅限关键字参数
* POSITIONAL_ONLY       仅限定位参数(目前 Python 声明函数的语法不支持，但有些使用 C 语言实现且不接受关键字参数的函数 如：divmod 支持)
除此之外 inspect.Parameter 还有一个 annotation (注解) 属性  
  
inspect.Signature 对象有个 bind 方法，可以把任意个参数绑定到签名中的形参上，所用的规则与实参到形参的匹配方式一样，框架可以使用这个方法在真则调用参数前验证参数

In [3]:
import inspect

def tag(name, *content, cls=None, **attrs):
    '''生成一个或多个 HTML 标签'''
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value)
                            for attr, value in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' % (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

sig = inspect.signature(tag)    # 获取 tag 函数的签名
my_tag = {'name': 'img', 'title': 'Sunset', 'src': 'Sunset.jpg', 'cls': 'framed'}
bound_args = sig.bind(**my_tag) # 把一个字典参数传给 .bind() 方法
bound_args

<BoundArguments (name='img', cls='framed', attrs={'title': 'Sunset', 'src': 'Sunset.jpg'})>

In [6]:
for name,value in bound_args.arguments.items():
    print(name, '=', value)

name = img
cls = framed
attrs = {'title': 'Sunset', 'src': 'Sunset.jpg'}


In [10]:
del my_tag['name']  # 删除必须指定的参数

KeyError: 'name'

In [12]:
bound_args = sig.bind(**my_tag) # 此时再次绑定抛出错误，缺少 name 参数

TypeError: missing a required argument: 'name'